In [1]:
from iqcc_calibration_tools.quam_config.components import Quam, Transmon
from iqcc_calibration_tools.random_exp.classical_shadow import ClassicalShadow, ShadowConfig, SYdgGate
from iqcc_calibration_tools.random_exp.xeb import QUAGateSet, QUAGate
from qualang_tools.units import unit
from qiskit.circuit import QuantumCircuit
from rl_qoc.qua.iqcc import *
from qm import generate_qua_script
import numpy as np

u = unit(coerce_to_integer=True)
backend_name = "qolab"
machine, iqcc = get_machine_from_iqcc(backend_name)
qubits  = machine.active_qubits
readout_qubit_indices = [0]
readout_qubits = [qubits[i] for i in readout_qubit_indices]
target_qubit_indices = [0]
target_qubits = [qubits[i] for i in target_qubit_indices]


def sx_macro(qubit: Transmon):
    qubit.xy.play("x90")
    
def sy_macro(qubit: Transmon):
    qubit.xy.play("-y90")
    
def z_macro(qubit: Transmon):
    qubit.wait(4)

def h_macro(qubit: Transmon):
    qubit.apply("x")
    qubit.xy.frame_rotation_2pi(0.5)

def input_state_macro(angle, **kwargs):
    q0 = target_qubits[0]
    q0.apply("x")
    # q1 = target_qubits[1]
    # qp = q0@q1
    # h_macro(q0)
    # qp.apply("cz")
    # h_macro(q1)
    # q1.apply("rz", angle)
    # h_macro(q1)
    # qp.apply("cz")
    # h_macro(q1)
   
def input_state_circuit(angle: float, **kwargs) -> QuantumCircuit:
    # qc = QuantumCircuit(2)
    # qc.h([0, 1])
    # qc.cx(0, 1)
    # qc.rz(angle, 1)
    # qc.cx(0, 1)
    qc = QuantumCircuit(1)
    qc.x(0)
    
    return qc

measurement_basis = {0: QUAGate("sx", sx_macro),
                     1: QUAGate(SYdgGate(), sy_macro),
                     2: QUAGate("z", z_macro)}

shadow_size = 10 # Number of shots/snapshots to construct the shadow
seed = 1234
np.random.seed(seed)
# Define custom snapshots here if needed (otherwise, sampling is done in real time)
gate_indices = np.random.randint(0, 3, (shadow_size, len(target_qubits)))
wait_duration = 0.1*u.us

input_macro_kwargs = {}
shadow_config = ShadowConfig(shadow_size=shadow_size,
                             shots_per_snapshot=128,
                            input_state_prep_macro=input_state_macro,
                            input_state_circuit=input_state_circuit,
                            measurement_basis=measurement_basis,
                            qubits=target_qubits,
                            readout_qubits=readout_qubits,
                            readout_pulse_name="readout",
                            reset_method="active", #"active",
                            reset_kwargs={"cooldown_time": 80*u.us,
                                          "max_tries": 5,
                                          "pi_pulse": "x180"},
                            input_state_prep_macro_kwargs=input_macro_kwargs,
                            # gate_indices=gate_indices,
                            # num_angles=100, # Specify number of angles to sample (creates: np.linspace(0, np.pi, num_angles))
                            seed=seed,
                            )

shadow_exp = ClassicalShadow(shadow_config, machine)

# print("Generating QUA script...")
# print(generate_qua_script(shadow_exp.cs_prog(simulate=False)))
job = shadow_exp.run()
# New axis: angle
# Each element in the results corresponds to a collection of snapshots for a given angle defining the input state.
results = job.result() # [({"010": 2, "110": 3, ...}, [0, 1, 2]), ({"101": 5, "100": 4, ...}, [2, 0, 1]), ...]
ideal_results = job.ideal_result()

print("Results:")
print(results)
print("Ideal results:")
print(ideal_results)

gate_dict = {i: qua_gate.gate for i, qua_gate in measurement_basis.items()}

# Result format: List of (bitstring, random_gate_indices) of size shadow_size

#TODO: Add post-processing for shadow tomography below


2025-10-15 11:01:34,010 - qm - INFO     - Starting session: 8151c9e2-7e60-49e2-8823-b8b3a89ff686


/Users/arthurostrauss/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Coding_projects/Quantum_Optimal_Control/rl_qoc/helpers/__init__.py:9: UserWarning: pulse_utils is not available
  warnings.warn("pulse_utils is not available")
/Users/arthurostrauss/Library/CloudStorage/OneDrive-QMMachinesLTD/GitHub/quam/quam/components/channels.py:636: UserWarning: The 'thread' element argument is deprecated from qm.qua >= 1.2.2. Use 'core' instead.
  warnings.warn(


2025-10-15 11:01:42,581 - qm - WARNING  - Could not generate a loaded config. Maybe there is no `QuantumMachinesManager` instance?
2025-10-15 11:01:42 INFO Could not generate a loaded config. Maybe there is no `QuantumMachinesManager` instance?


Output()

[11:01:43] QUA program submitted to qolab (id = b9ca2398-4d85-4172-ab19-4ca8c06748fe)              ]8;id=292047;file:///Users/arthurostrauss/anaconda3/envs/qua_libs_entropica/lib/python3.11/site-packages/iqcc_cloud_client/computers.py\computers.py]8;;\:]8;id=660951;file:///Users/arthurostrauss/anaconda3/envs/qua_libs_entropica/lib/python3.11/site-packages/iqcc_cloud_client/computers.py#324\324]8;;\

[11:01:44] Execution started                                                                       ]8;id=225899;file:///Users/arthurostrauss/anaconda3/envs/qua_libs_entropica/lib/python3.11/site-packages/iqcc_cloud_client/computers.py\computers.py]8;;\:]8;id=160097;file:///Users/arthurostrauss/anaconda3/envs/qua_libs_entropica/lib/python3.11/site-packages/iqcc_cloud_client/computers.py#344\344]8;;\

[11:01:50] Execution finished                                                                      ]8;id=611193;file:///Users/arthurostrauss/anaconda3/envs/qua_libs_entropica/lib/python3.11/site-packages/iqcc_cloud_client/computers.py\computers.py]8;;\:]8;id=881787;file:///Users/arthurostrauss/anaconda3/envs/qua_libs_entropica/lib/python3.11/site-packages/iqcc_cloud_client/computers.py#358\358]8;;\

QUA program has error:

/app/.venv/lib/python3.11/site-packages/qm/program/_qua_config_schema.py:1880: DeprecationWarning: 'version' is 
deprecated since "1.2.2" and will be removed in "1.3.0". Please remove it from the Qua config.
  warnings.warn(
/app/.venv/lib/python3.11/site-packages/qm/api/v2/qmm_api.py:159: DeprecationWarning: close_other_machines is not 
set, as from 1.3.0 default will be False, now setting to True. Please set it explicitly to remove this message and 
keep the wanted behavior in future versions.
  warnings.warn(

Results:
[(0.0, [({'0': 128, '1': 0}, array([[0],
       [1],
       [2],
       [1],
       [2],
       [1],
       [2],
       [1],
       [1],
       [2]])), ({'0': 128, '1': 0}, array([[1],
       [2],
       [0],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0]])), ({'0': 128, '1': 0}, array([[0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [0],
       [1],
       [0]])), ({'0': 128, '1': 0}, array([[1],
       [0],
       [0],
       [1],
       [2],
       [2],
       [1],
       [0],
       [0],
       [2]])), ({'0': 128, '1': 0}, array([[0],
       [0],
       [0],
       [2],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0]])), ({'0': 128, '1': 0}, array([[0],
       [1],
       [2],
       [0],
       [1],
       [2],
       [0],
       [0],
       [1],
       [0]])), ({'0': 128, '1': 0}, array([[1],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [0],
  

In [ ]:
job.angles[:,0]